# Analisis de Datos Financieros SENATEX

In [1]:
!pip install xlrd openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 24.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import os
import string

In [3]:
ruta_carpeta = 'databases/'
archivos_excel = [f for f in os.listdir(ruta_carpeta) if f.endswith('.xls') or f.endswith('.xlsx')]

In [4]:
%%time

dataframes = {}

for archivo in archivos_excel:
    ruta_completa = os.path.join(ruta_carpeta, archivo)
    xl = pd.ExcelFile(ruta_completa)

    # Identificar hojas relevantes para cada archivo
    hojas_relevantes = [nombre for nombre in xl.sheet_names if "RClaEntidades" in nombre]

    # Leer y combinar las hojas relevantes
    df_combined = pd.DataFrame()
    for hoja in hojas_relevantes:
        df_temp = xl.parse(hoja, header=None)
        df_combined = pd.concat([df_combined, df_temp], ignore_index=True)

    # Almacenar el DataFrame en el diccionario
    nombre_base = archivo.split('.')[0]
    dataframes[nombre_base] = df_combined

CPU times: user 451 ms, sys: 17.6 ms, total: 468 ms
Wall time: 1.07 s


In [5]:
%%time
df_pagado = dataframes['Pagos21']

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.72 µs


In [6]:
def reordenar_datos_dinamico(archivo_entrada):
    # Leer el archivo Excel
    datos = archivo_entrada

    # Encontrar todas las filas donde está 'Tipo Doc.'
    filas_tipo_doc = datos.index[datos[0] == 'Tipo Doc.'].tolist()
    len_datos = len(datos)

    # Calculando las filas anteriores a la siguiente 'Tipo Doc.'
    filas_previas_a_siguiente_tipo_doc = [filas_tipo_doc[i + 1] - 1 if i + 1 < len(filas_tipo_doc) else len_datos - 2 for i in range(len(filas_tipo_doc))]

    # Si no se encuentra 'Tipo Doc.', lanzar un error
    if not filas_tipo_doc:
        raise ValueError("No se encontró 'Tipo Doc.' en la columna especificada")

    # Creando una estructura para almacenar los datos extraídos para cada 'Tipo Doc.'
    datos_combinados = []

    for i, fila_tipo_doc in enumerate(filas_tipo_doc):
        fila_datos_primera = fila_tipo_doc + 1
        fila_datos_tercera = fila_tipo_doc + 3
        fila_datos_quinta = fila_tipo_doc + 5
        fila_datos_onceava = filas_previas_a_siguiente_tipo_doc[i]

        # Extraer los datos de cada fila especificada
        datos_primera_fila = {}

        if fila_datos_primera < len(datos):
            # Extraer datos de la primera fila
            datos_primera_fila = {
                "Tipo Doc.": datos.iloc[fila_datos_primera][0],
                "Tipo Reg.": datos.iloc[fila_datos_primera][2],
                "Ent.": datos.iloc[fila_datos_primera][5],
                "DA": datos.iloc[fila_datos_primera][7],
                "Nº Prev.": datos.iloc[fila_datos_primera][9],
                "Nº": datos.iloc[fila_datos_primera][12],
                "Nº Dev.": datos.iloc[fila_datos_primera][13],
                "Nº Pag.": datos.iloc[fila_datos_primera][14],
                "Sec.": datos.iloc[fila_datos_primera][15],
                "C. G.": datos.iloc[fila_datos_primera][16],
                "SISIN": datos.iloc[fila_datos_primera][17],
                "SIGADE": datos.iloc[fila_datos_primera][18],
                "Sist.": datos.iloc[fila_datos_primera][19],
                "Tipo Doc. Rdo.": datos.iloc[fila_datos_primera][20],
                "Nº Doc.": datos.iloc[fila_datos_primera][21],
                "Nº de Pago": datos.iloc[fila_datos_primera][22],
                "Fecha de Aprobación": datos.iloc[fila_datos_primera][23],
                "Fecha de Firma": datos.iloc[fila_datos_primera][25],
                "Total Autorizado": datos.iloc[fila_datos_primera][26],
                "Deducciones": datos.iloc[fila_datos_primera][27],
                "Multas": datos.iloc[fila_datos_primera][28],
                "Liquido Pagable": datos.iloc[fila_datos_primera][29],
                "Estado": datos.iloc[fila_datos_primera][30]
            }

        datos_tercera_fila = {}
        if fila_datos_tercera < len(datos):
            # Extraer datos de la tercera fila
            datos_tercera_fila = {
                "Fte.": datos.iloc[fila_datos_tercera][3],
                "Org.": datos.iloc[fila_datos_tercera][6],
                "Banco": datos.iloc[fila_datos_tercera][8],
                "Cuenta": datos.iloc[fila_datos_tercera][12],
                "Libreta": datos.iloc[fila_datos_tercera][15]
            }

        datos_onceava_fila = {}
        if fila_datos_onceava < len(datos):
            # Extraer datos de la onceava fila
            datos_onceava_fila = {
                "Resumen:": datos.iloc[fila_datos_onceava][4]
            }

        if fila_datos_quinta < len(datos):
            # Identificar el número de filas a extraer comenzando desde la quinta fila
            n = 0
            while fila_datos_quinta + n < filas_previas_a_siguiente_tipo_doc[i] -2 and not datos.iloc[fila_datos_quinta + n].isnull().all():
                # Crear una fila por cada conjunto de datos de la quinta fila
                datos_fila = datos_primera_fila.copy()
                datos_fila.update(datos_tercera_fila)
                datos_fila.update(datos_onceava_fila)
                datos_fila.update({
                    "UE": datos.iloc[fila_datos_quinta + n][3],
                    "Prog": datos.iloc[fila_datos_quinta + n][6],
                    "Proy": datos.iloc[fila_datos_quinta + n][8],
                    "Act": datos.iloc[fila_datos_quinta + n][10],
                    "Obj": datos.iloc[fila_datos_quinta + n][12],
                    "Ent. Trans.": datos.iloc[fila_datos_quinta + n][13],
                    "Importe Preventivo": datos.iloc[fila_datos_quinta + n][15],
                    "Importe Compromiso": datos.iloc[fila_datos_quinta + n][18],
                    "Importe Devengado": datos.iloc[fila_datos_quinta + n][21],
                    "Importe Pago": datos.iloc[fila_datos_quinta + n][24]
                })
                datos_combinados.append(datos_fila)
                n += 1
    # Convertir los datos combinados en un DataFrame
    df_combinado = pd.DataFrame(datos_combinados)

    return df_combinado

In [7]:
df_pagado_f = reordenar_datos_dinamico(df_pagado)

In [8]:
df_pagado_f.to_excel("Prueba.xlsx", index=False)